In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [11]:
def vgg19Model(image_size = (224, 224), if_draw_model = False):
    width = image_size[0]
    height = image_size[1]
    
    base_model = ResNet50(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = False
        
 #   for layer in model.layers[140:]:
 #       layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
        
    return model
    

In [19]:
import os
import shutil

def prepare_val():
    dirname = 'imgs/val'
    if os.path.exists(dirname):
        return
    os.mkdir(dirname)
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10']
    for sub_dir in sub_dirs:
        address = dirname.formate('{}{}')
        os.mkdir(sub_dir)
        filenames = os.listdir(sub_dir)
        print(filenames)
        return

prepare_val()
    

In [7]:
#训练
nb_train_samples = 22434
def train_model(Model, image_size=(224, 224),batch_size =64):
    model = Model()
    
    train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
    
    train_generator = train_datagen.flow_from_directory(
    'imgs/train',
    target_size=(image_size[0], image_size[1]),
    batch_size=batch_size,
    class_mode='categorical')
        
    model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_split = 0.1)
    
    model.save_weights('model.h5')
    
    return model


In [12]:
train_model(vgg19Model)

/Users/apple/.keras/models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
Found 22424 images belonging to 10 classes.


TypeError: fit_generator() got an unexpected keyword argument 'validation_split'